In [ ]:
# Luke Munday
# 199116045
# 3CB107 - AI Report

In [ ]:
import os, sys, shutil, random

In [ ]:
original_dataset = "Hands/"
filename = "HandInfo.txt"
dataset_folder_name = "dataset_final/"
hand_types = ['palm', 'dorsal', 'both']
mode = 0o777

In [ ]:
def create_dir(folder_path, hand_type='', gender=''):
    try: 
        os.makedirs(os.path.join(folder_path, hand_type, gender), mode)
    except OSError as error:
        print(error)

In [ ]:
# Takes a copy of the specified image from the Hands data set and moves it into the correct directory
def sort_image(original_dataset, dataset_folder_name, info_dict, phase_type):
    if ("palmar" in info_dict['hand_aspect']):
        shutil.copy2(original_dataset + info_dict['image_name'], os.path.join(dataset_folder_name, "palm" , phase_type, info_dict['gender']))
    elif ("dorsal" in info_dict['hand_aspect']):
        shutil.copy2(original_dataset + info_dict['image_name'], os.path.join(dataset_folder_name, "dorsal", phase_type, info_dict['gender']))
    shutil.copy2(original_dataset + info_dict['image_name'], os.path.join(dataset_folder_name, "both", phase_type, info_dict['gender']))

In [ ]:
# Calculates the 80-20 split and randomly samples subjects from the each of the passed in lists
# Then returns the train and validation list 
def get_gender_lists(gender_list):
    k = len(gender_list) * percentage / 100
    k = int(k)
    indicies = random.sample(range(len(gender_list)), k)
    train_list = [gender_list[i] for i in indicies]
    valid_list = list(set(gender_list)-set(train_list))
    
    return train_list, valid_list

In [ ]:
# id, age, gender, skinColor, accessories, nailPolish, aspectOfHand, imageName, irregularities
# Return a dict of meta data from the given line in the file
def get_metadata(line):
    line_info = line.split(",")
    info_dict = {
        'subject' : line_info[0],
        'gender' : line_info[2],
        'hand_aspect' : line_info[6],
        'image_name' : line_info[7]
    }
    return info_dict

In [ ]:
# Create the dirs for each of the hand types
for hand_type in hand_types:
    dataset_folder_path = os.path.join(dataset_folder_name, hand_type)
    create_dir(dataset_folder_name)
    create_dir(dataset_folder_path)

In [ ]:
male_list = []
female_list = []

f = open(filename, "r")

# Calculates which subjects are not currently present in each of the gender lists and adds them ready to split
for line in f:
    info_dict = get_metadata(line)
    if (info_dict['gender'] == "male"):
        if (info_dict['subject'] not in male_list):
            male_list.append(info_dict['subject'])
    else:
        if (info_dict['subject'] not in female_list):
            female_list.append(info_dict['subject'])

percentage = float(80)
training_male, valid_male = get_gender_lists(male_list)
training_female, valid_female = get_gender_lists(female_list)

In [ ]:
# Create 'male' and 'female' dirs inside each of the hand type's valid and train dirs
for hand_type in hand_types:
    dataset_folder_path = os.path.join(dataset_folder_name, hand_type)
    
    for gender in ['male', 'female']:
        create_dir(dataset_folder_path, "train", gender)
        create_dir(dataset_folder_path, "valid", gender)

In [ ]:
f = open(filename, "r")

# Go through every image in the Hands set and copy them to their new specified position in the training and validation dirs
for line in f:
    info_dict = get_metadata(line)

    if (info_dict['subject'] in training_female or info_dict['subject'] in training_male):
        sort_image(original_dataset, dataset_folder_name, info_dict, "train")
    elif (info_dict['subject'] in valid_female or info_dict['subject'] in valid_male):
        sort_image(original_dataset, dataset_folder_name, info_dict, "valid")